Farmer problem (here-and-now) policy

Mauricio Tombini, Jordan Jalving, and Victor M. Zavala 

University of Wisconsin-Madison, 2016

In [1]:
using JuMP 
using Ipopt 

In [2]:
# Model parameters

NS = 5;                  # number of scenarios
S = collect(1:NS);       # scenario set
P = collect(1:3);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data
yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;
yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

In [3]:
# Create Model (standard)
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, x[P] >= 0)    # acres devoted to crops
@variable(m, y[S,P] >= 0)    # crops purchase
@variable(m, w[S,P] >= 0)    # crops sold;
@variable(m, cost[s in S])   # per scenario cost

@constraint(m, varcost[s in S], cost[s] == sum(prcost[j]*x[j] + pcost[j]*y[s,j] - scost[j]*w[s,j] for j in P) )
@constraint(m, cap[s in S], sum(x[j] for j in P) <= 500)
@constraint(m, bal[s in S,j in P], yield[s,j]*x[j]+y[s,j]-w[s,j] >= demand[j]) 
@constraint(m, sellb[s in S], w[s,3] <= 6000)
@constraint(m, buyb[s in S], y[s,3] <= 0)

@objective(m, Min, (1/NS)*sum(cost[s] for s in S))

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [4]:
# Results
println(getvalue(x))
println("")
println(getvalue(y))
println("")
println(getvalue(w))
println("")
println(getvalue(cost))
println("")
println("obj ",getobjectivevalue(m))

x: 1 dimensions:
[1] = 220.00000380025003
[2] = 79.99999920015239
[3] = 200.00000199955198

y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 0.0
  [1,3] = 9.632874071974721e-9
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 9.749026913123416e-9
[3,:]
  [3,1] = 0.0
  [3,2] = 0.0
  [3,3] = 9.667054155588147e-9
[4,:]
  [4,1] = 0.0
  [4,2] = 0.0
  [4,3] = 9.709581102331193e-9
[5,:]
  [5,1] = 0.0
  [5,2] = 0.0
  [5,3] = 5.4591602227690745e-9

w: 2 dimensions:
[1,:]
  [1,1] = 350.0000114907356
  [1,2] = 0.0
  [1,3] = 2000.0000200148047
[2,:]
  [2,1] = 350.0000114907356
  [2,2] = 0.0
  [2,3] = 3000.0000300126812
[3,:]
  [3,1] = 350.0000114907356
  [3,2] = 0.0
  [3,3] = 4000.000040010359
[4,:]
  [4,1] = 350.0000114907356
  [4,2] = 0.0
  [4,3] = 5000.000050008161
[5,:]
  [5,1] = 350.0000114907356
  [5,2] = 0.0
  [5,3] = 6000.000059999594

cost: 1 dimensions:
[1] = -28100.001770977455
[2] = -64100.002130901
[3] = -100100.0024908174
[4] = -136100.0028507383
[5] = -172100.00321042986

obj -100100

In [5]:
# Create Model (non-anticipativity)
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, x[S,P] >= 0)    # acres devoted to crops
@variable(m, y[S,P] >= 0)    # crops purchase
@variable(m, w[S,P] >= 0)    # crops sold
@variable(m, cost[s in S])   # per scenario cost

@constraint(m, varcost[s in S], cost[s] == sum(prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j] for j in P)) 
@constraint(m, cap[s in S], sum(x[s,j] for j in P) <= 500)
@constraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@constraint(m, sellb[s in S], w[s,3] <= 6000)
@constraint(m, buyb[s in S], y[s,3] <= 0)
@constraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])

@objective(m, Min, (1/NS)*sum(cost[s] for s in S))

solve(m)

:Optimal

In [6]:
# Results
println(getvalue(x))
println("")
println(getvalue(y))
println("")
println(getvalue(w))
println("")
println(getvalue(cost))
println("")
println("obj ",getobjectivevalue(m))

x: 2 dimensions:
[1,:]
  [1,1] = 220.00000379973474
  [1,2] = 79.99999920015239
  [1,3] = 200.0000020000673
[2,:]
  [2,1] = 220.00000379973474
  [2,2] = 79.99999920015239
  [2,3] = 200.0000020000673
[3,:]
  [3,1] = 220.00000379973474
  [3,2] = 79.99999920015239
  [3,3] = 200.0000020000673
[4,:]
  [4,1] = 220.00000379973474
  [4,2] = 79.99999920015239
  [4,3] = 200.0000020000673
[5,:]
  [5,1] = 220.00000379973474
  [5,2] = 79.99999920015239
  [5,3] = 200.0000020000673

y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 0.0
  [1,3] = 0.0
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 0.0
[3,:]
  [3,1] = 0.0
  [3,2] = 0.0
  [3,3] = 0.0
[4,:]
  [4,1] = 0.0
  [4,2] = 0.0
  [4,3] = 0.0
[5,:]
  [5,1] = 0.0
  [5,2] = 0.0
  [5,3] = 0.0

w: 2 dimensions:
[1,:]
  [1,1] = 350.00001148944745
  [1,2] = 0.0
  [1,3] = 2000.0000200003253
[2,:]
  [2,1] = 350.00001148944745
  [2,2] = 0.0
  [2,3] = 3000.000030000662
[3,:]
  [3,1] = 350.00001148944745
  [3,2] = 0.0
  [3,3] = 4000.0000400009985
[4,:]
  [4,1] = 350